## LangChain-檢索增強生成(RAG)範例-Ollama
2025/06/29 蘇彥庭

* 本篇範例主要建立中國四大名著的RAG
    * 四大名著文本[下載位置](https://github.com/tennessine/corpus)
    * 四大名著:《三國演義》、《西遊記》、《水滸傳》、《紅樓夢》
* Text-embeddings Model採用`nomic-embed-text`(以Ollma來部署)
    * [Model資訊連結](https://ollama.com/library/nomic-embed-text)
    * Ollma下載模型指令: `ollama pull nomic-embed-text:v1.5`
* 聊天模型採用`gemma3:1b`(以Ollma來部署)
    * [Model資訊連結](https://ollama.com/library/gemma3)
    * Ollma下載模型指令: `ollama pull gemma3:1b`

In [3]:
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
import os

In [4]:
# 設定文本切割器
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

* 文本切割相關文章
    * [Langchain's Character Text Splitter - In-Depth Explanation](https://medium.com/@krishnahariharan/langchains-character-text-splitter-in-depth-explanation-5b0bf743121c)

In [5]:
# 讀取文本進行切分
# folder_path = "docs" 
# docs = []
# for f in os.listdir(folder_path):
#     file_path = os.path.join(folder_path, f)
#     if os.path.isfile(file_path) and f.endswith(".txt"):
#         loader = TextLoader(file_path, encoding="utf-8")
#         # 直接載入並切割後 append
#         docs.extend(text_splitter.split_documents(loader.load()))

filePath = "docs\三国演义.txt"
loader = TextLoader(filePath, encoding="utf-8")
docs = text_splitter.split_documents(loader.load())

print(f"已完成切割，共切出 {len(docs)} 個 chunks")

已完成切割，共切出 1749 個 chunks


In [6]:
# 顯示切割結果(取前5篇)
for i, doc in enumerate(docs[0:5]):
    print(f"【Chunk {i} - {doc.metadata.get('source')}】\n{doc.page_content}\n")

【Chunk 0 - docs\三国演义.txt】
《三国演义》罗贯中

第一回 宴桃园豪杰三结义 斩黄巾英雄首立功

    		

    滚滚长江东逝水，浪花淘尽英雄。是非成败转头空。

    青山依旧在，几度夕阳红。白发渔樵江渚上，惯

    看秋月春风。一壶浊酒喜相逢。古今多少事，都付

    笑谈中。

    ——调寄《临江仙》

    话说天下大势，分久必合，合久必分。周末七国分争，并入于秦。及秦灭之后，楚、汉分争，又并入于汉。汉朝自高祖斩白蛇而起义，一统天下，后来光武中兴，传至献帝，遂分为三国。推其致乱之由，殆始于桓、灵二帝。桓帝禁锢善类，崇信宦官。及桓帝崩，灵帝即位，大将军窦武、太傅陈蕃共相辅佐。时有宦官曹节等弄权，窦武、陈蕃谋诛之，机事不密，反为所害，中涓自此愈横。

【Chunk 1 - docs\三国演义.txt】
建宁二年四月望日，帝御温德殿。方升座，殿角狂风骤起。只见一条大青蛇，从梁上飞将下来，蟠于椅上。帝惊倒，左右急救入宫，百官俱奔避。须臾，蛇不见了。忽然大雷大雨，加以冰雹，落到半夜方止，坏却房屋无数。建宁四年二月，洛阳地震；又海水泛溢，沿海居民，尽被大浪卷入海中。光和元年，雌鸡化雄。六月朔，黑气十余丈，飞入温德殿中。秋七月，有虹现于玉堂；五原山岸，尽皆崩裂。种种不祥，非止一端。帝下诏问群臣以灾异之由，议郎蔡邕上疏，以为蜺堕鸡化，乃妇寺干政之所致，言颇切直。帝览奏叹息，因起更衣。曹节在后窃视，悉宣告左右；遂以他事陷邕于罪，放归田里。后张让、赵忠、封谞、段珪、曹节、侯览、蹇硕、程旷、夏恽、郭胜十人朋比为奸，号为“十常侍”。帝尊信张让，呼为“阿父”。朝政日非，以致天下人心思乱，盗贼蜂起。

【Chunk 2 - docs\三国演义.txt】
时巨鹿郡有兄弟三人，一名张角，一名张宝，一名张梁。那张角本是个不第秀才，因入山采药，遇一老人，碧眼童颜，手执藜杖，唤角至一洞中，以天书三卷授之，曰：“此名《太平要术》，汝得之，当代天宣化，普救世人；若萌异心，必获恶报。”角拜问姓名。老人曰：“吾乃南华老仙也。”言讫，化阵清风而去。角得此书，晓夜攻习，能呼风唤雨，号为“太平道人”。中平元年正月内，疫气流行，张角散施符水，为人治病，自称“大贤良师”。角有徒弟五百余人，云游四方，皆能书符念咒。次后徒众日多，角乃立三十六方，大方万余人，小方六七千，各立

In [7]:
# 建立 Embeddings
embedding_model_name = "nomic-embed-text:v1.5"
embeddings = OllamaEmbeddings(model=embedding_model_name)

In [8]:
# Chunk做Embeddings的範例(取前5篇)
texts = [doc.page_content for doc in docs[0:5]]
vectors = embeddings.embed_documents(texts)
for i in range(len(texts)):
    print(f"\n=== Chunk {i+1} ===")
    print(texts[i][:200])   # 顯示 chunk 前 200 字
    print(f"向量長度: {len(vectors[i])}")
    print(f"向量前 10 維: {vectors[i][:10]}")


=== Chunk 1 ===
《三国演义》罗贯中

第一回 宴桃园豪杰三结义 斩黄巾英雄首立功

    		

    滚滚长江东逝水，浪花淘尽英雄。是非成败转头空。

    青山依旧在，几度夕阳红。白发渔樵江渚上，惯

    看秋月春风。一壶浊酒喜相逢。古今多少事，都付

    笑谈中。

    ——调寄《临江仙》

    话说天下大势，分久必合，合久必分。周末七国分争，并入于秦。及秦灭之后，楚、汉分争，又并入于
向量長度: 768
向量前 10 維: [0.011270066, 0.025355915, -0.16098003, -0.031632185, 0.052455347, 0.057537183, 0.0015493581, -0.018163009, -0.006354736, 0.013061971]

=== Chunk 2 ===
建宁二年四月望日，帝御温德殿。方升座，殿角狂风骤起。只见一条大青蛇，从梁上飞将下来，蟠于椅上。帝惊倒，左右急救入宫，百官俱奔避。须臾，蛇不见了。忽然大雷大雨，加以冰雹，落到半夜方止，坏却房屋无数。建宁四年二月，洛阳地震；又海水泛溢，沿海居民，尽被大浪卷入海中。光和元年，雌鸡化雄。六月朔，黑气十余丈，飞入温德殿中。秋七月，有虹现于玉堂；五原山岸，尽皆崩裂。种种不祥，非止一端。帝下诏问群臣以灾异之由，
向量長度: 768
向量前 10 維: [0.009261503, 0.021552268, -0.18155636, -0.007987923, -0.020716637, 0.00804173, 0.06410436, -0.03425317, -0.0022806493, -0.042737152]

=== Chunk 3 ===
时巨鹿郡有兄弟三人，一名张角，一名张宝，一名张梁。那张角本是个不第秀才，因入山采药，遇一老人，碧眼童颜，手执藜杖，唤角至一洞中，以天书三卷授之，曰：“此名《太平要术》，汝得之，当代天宣化，普救世人；若萌异心，必获恶报。”角拜问姓名。老人曰：“吾乃南华老仙也。”言讫，化阵清风而去。角得此书，晓夜攻习，能呼风唤雨，号为“太平道人”。中平元年正月内，疫气流行，张角散施符水，为人治病，自称“大贤良师”。角
向量長度: 768
向量前 10 維: [-0.0033749824, 0

In [9]:
# 建立向量資料庫
persist_directory = "./chroma_db_ollama"
if os.path.exists(os.path.join(persist_directory, "index")):
    print("偵測到已有 Chroma 向量資料庫，直接載入...")
    db = Chroma(
        persist_directory=persist_directory,
        embedding_function=embeddings
    )
else:
    print("未偵測到 Chroma 向量資料庫，建立新向量資料庫...")
    db = Chroma.from_documents(
        docs,
        embeddings,
        persist_directory=persist_directory
    )

未偵測到 Chroma 向量資料庫，建立新向量資料庫...


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [10]:
# 設定檢索器
# k: 每次檢索時 從向量資料庫中取回最相關的 Top-k 筆文件段落(chunks)
retriever = db.as_retriever(search_kwargs={"k": 5})

In [11]:
# 建立 RAG QA Chain
llm_model_name = "gemma3:1b"
llm = Ollama(
    model=llm_model_name, 
    temperature=0.2  # 溫度(temperature)控制生成回答時的隨機性，介於0(保守)至1(隨機)
)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", # 將所有檢索到的段落「直接拼接」在一起，與問題一起輸入 LLM（最簡單、適合段落少時使用）
    retriever=retriever,
    return_source_documents=True
)

C:\Users\User\AppData\Local\Temp\ipykernel_9908\4039160444.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(


* RetrievalQA的chain_type參考文章
    * [Using langchain for Question Answering on Own Data](https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed)  

In [14]:
# LangCahin RetrievalQA 的Prompt Template
qa.combine_documents_chain.llm_chain.prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:")

LangCahin RetrievalQA 的Prompt Template:

```python
PromptTemplate(
    input_variables=['context', 'question'], 
    input_types={}, partial_variables={}, 
    template="Use the following pieces of context to answer the question at the end. " \
    "If you don't know the answer, just say that you don't know, don't try to make up an answer." \
    "\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"
    )
```

In [34]:
# 提問 & 回覆
# query = "請問劉備底下有哪些武將?請用繁體中文回答。"
# query = "張飛的武器是什麼?請用繁體中文回答。"
query = "誰對曹操的評價是'治世之能臣，亂世之奸雄'?請用繁體中文回答。"
result = qa(query)

print("\n=== 回覆 ===")
print(result["result"])

print("\n=== 檢索來源 (僅呈現前300字) ===")
for idx, doc in enumerate(result["source_documents"]):
    print(f"\n--- Source {idx+1} ({doc.metadata.get('source')}) ---")
    print(doc.page_content[:300])


=== 回覆 ===
根據提供的文本，曹操被評價為「治世之能臣，亂世之奸雄」。

**理由：**

文本中多次提到曹操的才華和能力，例如：「曹操說出甚話來，乃严峻低头丧气而不能对。」、「曹操说出甚话来，且听下文分解。」、「曹操说出甚话来，且听下文分解。」，以及对他的军事才能和政治智慧的赞赏。因此，文本中反复提及的评价都指向他是一位卓越的政治家和军事家，能够有效地治理国家，并以其卓越的才能在乱世中取得胜利。


=== 檢索來源 (僅呈現前300字) ===

--- Source 1 (docs\三国演义.txt) ---
乱；思善则生治，理之常也，故周文养民，以少取多；句践恤众，以弱毙强。此其术也。或曰：曩者楚强汉弱，约分鸿沟，张良以为民志既定则难动也，率兵追羽，终毙项氏；岂必由文王、句践之事乎？曰：商、周之际，王侯世尊，君臣久固。当此之时，虽有汉祖，安能仗剑取天下乎？及秦罢侯置守之后，民疲秦役，天下土崩，于是豪杰并争。今我与彼，皆传国易世矣，既非秦末鼎沸之时，实有六国并据之势，故可为文王，难为汉祖。时可而后动，数合而后举，故汤、武之师，不再战而克，诚重民劳而度时审也。如遂极武黩征，不幸遇难，虽有智者，不能谋之矣。”姜维看毕，大怒曰：“此腐儒之论也！”掷之于地，遂提川兵来取中原。乃问傅佥曰：“以公度之，可出何地

--- Source 2 (docs\三国演义.txt) ---
座上一人忽曰：“孔明所言，皆强词夺理，均非正论，不必再言。且请问孔明治何经典？”孔明视之，乃严酸也。孔明曰：“寻章摘句，世之腐儒也，何能兴邦立事？且古耕莘伊尹，钓渭子牙，张良、陈平之流。邓禹、耿弇之辈，皆有匡扶宇宙之才，未审其生平治何经典。岂亦效书生，区区于笔砚之间，数黑论黄，舞文弄墨而已乎？”严峻低头丧气而不能对。

--- Source 3 (docs\三国演义.txt) ---
发耳。但当速发雷霆，行权立断，则天人顺之。却反外檄大臣，临犯京阙，英雄聚会，各怀一心：所谓倒持干戈，授人以柄，功必不成，反生乱矣。”何进笑曰：“此懦夫之见也！”傍边一人鼓掌大笑曰：“此事易如反掌，何必多议！”视之，乃曹操也。正是：欲除君侧宵人乱，须听朝中智士谋。不知曹操说出甚话来，且听下文分解。

--- Source 4 (docs\三国演义.txt) ---
孙策既死，孙权哭倒于床前。张昭曰：“此非将军